##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

In [2]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.09615248,  0.22434415,  0.00839302,  0.5853356 , -0.25610363,
         0.33563435, -0.3481072 , -0.38838178, -1.0069251 , -0.16033137]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.09327527, 0.12851577, 0.10355476, 0.18438816, 0.07948782,
        0.14364456, 0.07250099, 0.06963906, 0.03751656, 0.08747702]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.5497844

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 888us/step - loss: 0.2957 - accuracy: 0.9134
Epoch 2/5
1875/1875 [==============================] - 2s 917us/step - loss: 0.1431 - accuracy: 0.9572
Epoch 3/5
1875/1875 [==============================] - 2s 927us/step - loss: 0.1068 - accuracy: 0.9669
Epoch 4/5
1875/1875 [==============================] - 2s 887us/step - loss: 0.0878 - accuracy: 0.9734
Epoch 5/5
1875/1875 [==============================] - 2s 873us/step - loss: 0.0734 - accuracy: 0.9766


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0733 - accuracy: 0.9783


[0.07334592193365097, 0.9782999753952026]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.4301469e-09, 3.6790309e-09, 7.7353789e-06, 3.8639770e-03,
        1.4534053e-10, 6.5805477e-07, 2.7205853e-14, 9.9611050e-01,
        2.4102815e-07, 1.6909064e-05],
       [8.1047495e-11, 5.2634912e-04, 9.9946886e-01, 4.7762087e-06,
        1.0279636e-14, 1.9449942e-08, 1.6602554e-09, 1.3959155e-11,
        1.9581677e-08, 7.6596901e-15],
       [3.9101710e-07, 9.9852115e-01, 1.4271392e-04, 3.5729248e-05,
        8.4919620e-06, 3.3824999e-05, 8.9435125e-06, 3.7416091e-04,
        8.7426213e-04, 3.1935775e-07],
       [9.9891818e-01, 7.7604668e-08, 1.4700084e-04, 2.3598004e-07,
        4.3221669e-05, 7.5931450e-05, 7.2281930e-04, 7.8348865e-05,
        9.6610461e-07, 1.3307797e-05],
       [1.8390895e-06, 1.4786753e-09, 4.1456296e-06, 5.7193898e-07,
        9.9552178e-01, 2.1304969e-07, 3.0548711e-07, 1.2864456e-04,
        1.8463807e-06, 4.3406347e-03]], dtype=float32)>